# TG_IR

## Before running, you have to supply a 'home' path (location of notebook) and a 'dir_data' path (location of raw data) in the first cell.
#### The program assumes a predefined directory structure that contains the supplied files:

+ **'home'**
    + *TG_IR.ipynb*
    + **Kali**
        + *cali.xlsx*
    + **Fitting**
    + **Sample_Listen**
    + **Parameter**
        + *labels.xlsx*
        + *surfgr_C.xlsx*
    + **Output**
        + **Plots**

Legend:
**directory**,*file*

In [1]:
from decimal import Decimal
import numpy as np

import pandas as pd

%matplotlib notebook
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.gridspec as grid
plt.rcParams.update({'font.size': 18})
plt.rcParams['figure.figsize'] = 15,7.5


import scipy as sp
from scipy.signal import find_peaks
from scipy.optimize import curve_fit, minimize

from sklearn import linear_model

import os
import sys
import datetime as dt
import pickle

import copy
import re

#directories for loading and saving of data
home=os.path.abspath('')#<<----------------------------------------
dir_data=os.path.abspath('')#<<----------------------------------------
dir_cali=os.path.join(home,'Kali')
dir_fits=os.path.join(home,'Fitting')
dir_samples=os.path.join(home,'Sample_Listen')
dir_parameters=os.path.join(home,'Parameter')
dir_output=os.path.join(home,'Output')
dir_plots=os.path.join(dir_output,'Plots')
os.chdir(home)
        
#plotlabels
inp=pd.read_excel(os.path.join(dir_parameters,'labels.xlsx'),sheet_name=['samples','gases','plots'],index_col=0)
LABELS=dict(zip(inp['samples'].index,inp['samples']['label']))
LABELS.update(dict(zip(inp['gases'].index,inp['gases']['label'])))

In [2]:
def get_label(key):
    if key in LABELS:
        return LABELS[key]
    try:
        if int(key)in LABELS:
            return LABELS[int(key)]
    except:
        return str(key)

In [3]:
def time():
    return str(dt.datetime.now().date())+'_'+str(dt.datetime.now().hour)+'-'+str(dt.datetime.now().minute).zfill(2)+'-'+str(dt.datetime.now().second).zfill(2)+'_'

In [5]:
def find_path(file,parent_dir):
    for dirpath,dirnames,filenames in os.walk(parent_dir):
        for filename in filenames:
            if filename[:filename.rfind('.')].find(file)!=-1:
                return dirpath

In [32]:
def find_files(file,suffix,parent_dir):
    files=[]
    for dirpath,dirnames,filenames in os.walk(parent_dir):
        for filename in filenames:
            if (re.search(re.escape(file),filename)!=None) and (re.search(suffix+'$',filename,flags=re.IGNORECASE)):
                files.append(os.path.join(dirpath,filename))   
    return files

# Plotting
```plot_TGA``` takes TGA-data and plots the sample mass against the sample temperature.

```plot_FTIR``` takes the FTIR-data and plots the absorbance of each gas over the time.

```FTIR_to_DTG``` reconstructs the DTG from the gas release and compares it to the DTG from TGA.

```plot_gauss```  takes lists of parameters of the normal distribution and plots the distributions seperately in the same plot.

In [1]:
def plot_TGA(TG_IR,plot,save=False,x_axis='Ts',y_axis='orig'):
    fig, TGA=plt.subplots()
    
    fig.subplots_adjust(right=.8)
    
    DTG=TGA.twinx()
    
    x=TG_IR.tga[x_axis]
    if y_axis=='rel':
        y=(TG_IR.tga[plot]/TG_IR.tga['mass'][0])*100
        yDTG=TG_IR.tga['dtg']*60/TG_IR.tga['mass'][0]
        ylabelDTG=r'DTG /% $min^{-1}$'
        if plot=='mass':
            ylabel='mass /%'
        elif plot=='heat_flow':
            ylabel='heat flow  $/mW mg^{-1}$'
        
    elif y_axis=='orig':
        y=TG_IR.tga[plot]
        yDTG=TG_IR.tga['dtg']*60
        ylabelDTG='DTG $/mg min^{-1}$'
        if plot=='mass':
            ylabel='mass /mg'
        elif plot=='heat_flow':
            ylabel='heat flow /mW'

    if x_axis=='Ts':
        TGA.set_xlabel('T /°C')
    elif x_axis=='t':
        x=x/60
        TGA.set_xlabel('t /min')
        temp=TGA.twinx()
        temp.plot(x,TG_IR.tga['Ts'],label='T /°C',ls='dashed',color='black')
        temp.spines['right'].set_position(('axes',1.15))
        temp.set_ylabel('T /°C')
        temp.legend(loc=1)
        
    gTGA,=TGA.plot(x,y,'r-',label=ylabel)
    gDTG,=DTG.plot(x,yDTG,'b--',label='DTG')
    
    TGA.set_ylabel(ylabel)
    DTG.set_ylabel(ylabelDTG)
        
    TGA.yaxis.label.set_color(gTGA.get_color())
    DTG.yaxis.label.set_color(gDTG.get_color())
    
    graphs=[gTGA,gDTG]
    
    TGA.set_title('{} - {}, {:.2f} mg'.format(get_label(TG_IR.info['sample']),TG_IR.info['run'],TG_IR.info['mass']))
    plt.show()
    
    if save==True:
        fig.savefig(os.path.join(dir_plots,'TGA','{}_{}_TG.png'.format(get_label(TG_IR.info['sample']),TG_IR.info['run'])), bbox_inches='tight', dpi=300)

def plot_FTIR(TG_IR,save=False,gases=[],x_axis='Ts',y_axis='orig'):
    colors =plt.rcParams['axes.prop_cycle'].by_key()['color']

    x=TG_IR.ir[x_axis]
    if gases==[]:
        gases=TG_IR.gases
    if y_axis=='rel':
        gases=TG_IR.linreg.index
        
    #setup figure and plot first gas
    fig, g1=plt.subplots()
    fig.subplots_adjust(right=.8)
    #
    if x_axis=='t':
        x=x/60
        g1.set_xlabel('t /min')
    elif x_axis=='Ts':
        g1.set_xlabel('T /°C')    
    if y_axis=='orig':
        g1.set_ylabel(get_label(gases[0])+' /AU')
        g1.yaxis.label.set_color(colors[0])
        g1.plot(x,TG_IR.ir[gases[0]])
    elif y_axis=='rel':
        g1.set_ylabel('mmol$g^{-1}s^{-1}$')
        g1.plot(x,TG_IR.ir[gases[0]],label=get_label(gases[0]))

    
    #append secondary, third... y-axis on right side
    graph=[] 
    props=[]
    for i,gas in enumerate(gases[1:]):
        if y_axis=='orig':
            y=TG_IR.ir[gas]
            
            graph.append(g1.twinx())
            graph[i].spines['right'].set_position(('axes',1+i*.1))
            graph[i].plot(x,y, color=colors[i+1])
            graph[i].set_ylabel(get_label(gas)+' /AU')
            graph[i].yaxis.label.set_color(colors[i+1])
            
        elif y_axis=='rel':
            tot_area=np.sum(TG_IR.ir[gas])
            tot_mol=(tot_area-TG_IR.linreg['intercept'][gas])/TG_IR.linreg['slope'][gas]*1000/TG_IR.info['dry_mass']
            y=TG_IR.ir[gas]/tot_area*tot_mol
            g1.plot(x,y,label=get_label(gas))

    plt.legend()
    g1.set_title('{} - {}, {:.2f} mg'.format(get_label(TG_IR.info['sample']),TG_IR.info['run'],TG_IR.info['mass']))
    plt.show()
    
    if save==True:
        fig.savefig(os.path.join(dir_plots,'IR','{}_{}_IR.png'.format(get_label(TG_IR.info['sample']),TG_IR.info['run'])), bbox_inches='tight', dpi=300)
        
def FTIR_to_DTG(TG_IR,x_axis='Ts',save=False,gases=[]):
    if gases==[]:
        gases=TG_IR.linreg.index

    Stoffdaten=pd.DataFrame({'co2':44.01,'co':28.01,'h2o':18.02,'caox':128.10},index=['M'])
    data=pd.merge(TG_IR.tga,TG_IR.ir,how='left',on=['t','Ts']).dropna()
    DTG=-sp.signal.savgol_filter(data['mass'], 13, 3, deriv=1)
    
    x=data[x_axis]
    y=np.zeros((len(gases),len(TG_IR.ir)))
    out=pd.DataFrame()
    out['t']=data['t']
    out['Ts']=data['Ts']
    cumul=np.zeros(len(TG_IR.ir))
    for i,gas in enumerate(gases):
        tot_area=np.sum(TG_IR.ir[gas])
        tot_mass=(tot_area-TG_IR.linreg['intercept'][gas])/TG_IR.linreg['slope'][gas]*Stoffdaten[gas]['M']
        y[i][:]=TG_IR.ir[gas]/tot_area*tot_mass
        out[gas]=y[i][:]
        cumul+=y[i][:]
   
    #setup
    fig=plt.figure(constrained_layout=True)
    gs = fig.add_gridspec(8, 1)
    stack= fig.add_subplot(gs[:-1, 0])
    stack.set_title('{} - {}, {:.2f} mg'.format(get_label(TG_IR.info['sample']),TG_IR.info['run'],TG_IR.info['mass']))
    error = fig.add_subplot(gs[-1,0],sharex=stack)

    #plotting of fit

    if x_axis=='Ts':
        stack.set_xlabel('T /°C')
        error.set_xlabel('T /°C')
    elif x_axis=='t':
        x=x/60
        stack.set_xlabel('t /min')
        error.set_xlabel('t /min')
        temp=stack.twinx()
        temp.plot(x,data['Ts'],ls='dashed',color='black',label='T')
        temp.set_ylabel('T /°C')
        temp.legend(loc=1)
        
    stack.stackplot(x,y,labels=[get_label(gas) for gas in gases])
    stack.plot(x,DTG,label='DTG')
    stack.set_ylabel('DTG, {} /$mg s^{{-1}}$'.format((', '.join([get_label(gas) for gas in gases]))))
    stack.legend(loc=2)
    error.plot(x,DTG-cumul)
    error.hlines(0,min(x),max(x),ls='dashed')
    
    error.set_ylabel('$\Delta$ DTG')
    fig.show()
    if save==True:
        fig.savefig(os.path.join(dir_plots,'IRDTG','{}_IRDTG.png'.format(TG_IR.info['name'])), bbox_inches='tight', dpi=300)
        out['dtg']=DTG
        out.to_excel(os.path.join(dir_output,TG_IR.info['name']+'_IRDTG.xlsx'))
    
def plot_gauss(x,*args):
    n=int(len(args)/3)
    A=args[:n]
    mu=args[n:2*n]
    sig=args[2*n:len(args)]
    for i in range(len(A)):
        y=gaussian(x,A[i],mu[i],sig[i])
        plt.plot(x,y)

# I/O

```corr_TGA``` takes TGA-data and corrects it by the buoyancy blank value.

```corr_FTIR``` takes FTIR-data and corrects the $CO_{2}$-signal. The signal has a consistant fluctuation, even when there is no analyte present due to the pressure fluctuations produced by the adsorption dryer, that is used to flush the FTIR constanly. A baseline measurement without analyte is shifted in both dimensions, so that it is in phase with the fluctuation in the data and then substracted from it and the minimum is set to zero by substracting the former minimum.

```baseline_als``` was taken from https://stackoverflow.com/questions/29156532/python-baseline-correction-library and based on the paper *Baseline Correction with Asymmetric Least Squares Smoothing* by Paul H. C. Eilers Hans F.M. Boelens
October 21, 2005.

```const_baseline``` calculates a constant baseline value that, when subtracted, minimizes the influence of noise on the quantification.

In [2]:
def corr_TGA(TGA,file_baseline,plot=False):
    path_baseline=find_files(file_baseline,'.txt',dir_data)[0]
    #opens the buoyancy blank value 'baseline' and substracts them from the original data 
    try:
        blindwert=pd.read_csv(path_baseline, delim_whitespace=True,decimal=',' ,names=['Index','t','Ts','Tr','mass'],skiprows=13, skipfooter=11,converters={'mass': lambda x: float(x.replace(',','.'))}).drop(columns='Index')
    except:
        print('>',path_baseline,' konnte nicht gefunden werden.')
        return None
    
    corr_data=TGA.copy()
    corr_data['mass']=corr_data['mass'].subtract(blindwert['mass'][:len(TGA)])
    
    #plotting of data, baseline and corrected value
    if plot==True:
        fig=plt.figure()
        
        x=TGA['Ts']
        y=TGA['mass']
        plt.plot(x,y,label='data')
        plt.plot(x,blindwert['mass'][:len(TGA)],label='baseline')
        plt.plot(x,corr_data['mass'],label='corrected')
        plt.xlabel('T /°C')
        plt.ylabel(y.name+' /mg')
        plt.legend()
        plt.title('TGA baseline correction')
        plt.show()
        
        
    return corr_data

def corr_FTIR(FTIR,file_baseline,save=False,plot=False):
    #opens FTIR data of the baseline and takes the 'co2' column
    corr_data=FTIR.copy()
    try:
        baseline=read_FTIR(file_baseline)
        baseline=np.array(baseline['co2'])

        #in the baseline the peaks and valleys as well as the amplitude of the baseline are determined
        peaks_baseline,properties_baseline=find_peaks(baseline,height=[None,None])#,height=[tol*min(baseline),tol*max(baseline)])
        valleys_baseline,valley_properties_baseline=find_peaks(-baseline,height=[None,None])#,height=[tol*min(-baseline),tol*max(-baseline)])
        amplitude_baseline=np.mean(properties_baseline['peak_heights'])+np.mean(valley_properties_baseline['peak_heights'])

        #in the original data the peaks and valleys that have similar height as the baseline are determined
        tol=1.5
        peaks,properties=find_peaks(FTIR['co2'],height=[-tol*amplitude_baseline,tol*amplitude_baseline])
        valleys,valley_properties=find_peaks(-FTIR['co2'],height=[None,None],prominence=amplitude_baseline*.05) 

        #the median distance between between baseline-peaks, the period is determined
        dist_peaks=np.diff(peaks_baseline)
        len_period=int(np.median(dist_peaks))

        #determination of the phase shift in x direction by checking if there is also a valley in the baseline in proximity
        #the x shift is calculated as the median of the differences
        dists=[]
        j=0
        for valley in valleys:
            while j<len(valleys_baseline)-1 and (valleys_baseline[j]-valley <=0):
                j=j+1
            if valleys_baseline[j]-valley>=0:
                dists.append(valleys_baseline[j]-valley)

        x_shift=int(sp.stats.mode(dists)[0])

        ###modifying of the baseline  
        #elongating the baseline by one period
        periode=baseline[:len_period]
        co2_baseline=np.concatenate((periode,baseline), axis=None)

        #shifting the baseline in x direction
        c=[]
        for x_offs in range(-1,len_period%x_shift+1):
            peaks,props=find_peaks(FTIR['co2']-co2_baseline[x_shift+x_offs:len(FTIR)+x_shift+x_offs],height=[None,None],prominence=amplitude_baseline*.02)
            c.append(len(peaks))
        x_offs=np.where(c==np.min(c))[0][0]-1

        co2_baseline=co2_baseline[x_shift+x_offs:len(FTIR)+x_shift+x_offs]
        
    except:
        print('No CO2 baseline found.')
        co2_baseline=np.zeros(len(FTIR))

    h2o_baseline=np.zeros(len(FTIR))
    
    ##return value
    
    corr_data['co']=corr_data['co'].subtract(min(corr_data['co']))
    corr_data['co']=corr_data['co'].subtract(const_baseline(corr_data['co'],0.002))
    corr_data['co2']=corr_data['co2'].subtract(co2_baseline)
    corr_data['co2']=corr_data['co2'].subtract(min(corr_data['co2']))
    corr_data['co2']=corr_data['co2'].subtract(const_baseline(corr_data['co2'],0.07))
    corr_data['h2o']=corr_data['h2o'].subtract(h2o_baseline)
    corr_data['h2o']=corr_data['h2o'].subtract(min(corr_data['h2o']))
    corr_data['h2o']=corr_data['h2o'].subtract(const_baseline(corr_data['h2o'],0.001))
    
    
    ###plotting of baseline, data and the corrected data
    if plot==True:
        
        fig=plt.figure()
        try:
            x=FTIR['Ts']
        except:
            x=FTIR['t']
        y=co2_baseline

        
        plt.plot(x,FTIR['co2'],label='data')
        plt.plot(x,baseline[:len(x)], label='baseline')
        plt.plot(x,y,label='corr. baseline')#,alpha=.5)
        plt.plot(x,corr_data['co2'],label='corr. data')
        plt.hlines(0,min(x),max(x),ls='dashed')
        
        plt.vlines(x.iloc[valleys],min(co2_baseline),max(FTIR['co2']-y),linestyle='dashed')
        plt.legend()
        if x.name=='t':    
            plt.xlabel(x.name+' /min')
        elif x.name=='Ts':    
            plt.xlabel('T /°C')
        plt.ylabel('$CO_2$ /AU')
        plt.title('$CO_2$ baseline correction')
        fig.savefig(os.path.join(dir_plots,'IR','IR_corr.png'), bbox_inches='tight', dpi=300)
        plt.show()
        
        fig=plt.figure()
        
        y=h2o_baseline
        plt.plot(x,FTIR['h2o'],label='data')
        plt.plot(x,y,label='baseline')
        plt.plot(x,corr_data['h2o'],label='corr. data')
        plt.hlines(0,min(x),max(x),ls='dashed')
        
        plt.legend()
        if x.name=='t':    
            plt.xlabel(x.name+' /min')
        elif x.name=='Ts':    
            plt.xlabel('T /°C')
        plt.ylabel('AU')
        plt.title('$H_2O$ baseline correction')
        plt.show()

    
    if save==True:
        out=pd.DataFrame()
        out['t']=x
        out['data']=FTIR['co2']
        out['baseline']=baseline[:len(x)]
        out['corr_baseline']=co2_baseline
        out['corr_data']=corr_data['co2']
        out.to_excel('baseline.xlsx')
    return corr_data

def baseline_als(y, lam=1e6, p=0.01, niter=10): #https://stackoverflow.com/questions/29156532/python-baseline-correction-library
    L = len(y)
    D = sp.sparse.csc_matrix(np.diff(np.eye(L), 2))
    w = np.ones(L)
    for i in range(niter):
        W = sp.sparse.spdiags(w, 0, L, L)
        Z = W + lam * D.dot(D.transpose())
        z = sp.sparse.linalg.spsolve(Z, w*y)
        w = p * (y > z) + (1-p) * (y < z)
    return z

def const_baseline(data,thres):
    baseline=data[data<thres]
    
    if len(baseline)==0:
        return 0
    else:
        return np.sum(baseline)/len(baseline)


```dry_weight``` takes the TGA data and returns the dry-weight of the sample as well as the dry mass loss. The moment where the sample is dry is derived from the *d²TG*-signal, where the first peak is assumed to be from the the desorption of physisorbed water.

```read_TGA``` opens the TGA-file in the given path and returns its content as a table.

```read_FTIR``` opens the FTIR-files in the given path and returns a table containing the absorbance at each given time.

In [3]:
####dry_weight sollte masse des wassers aus IR abziehen? gas adsorption ist dann im gewicht enthalten

def dry_weight(TG_IR,plot=False):
    #calculating and smoothing of DTG
    window_length=51
    polyorder=3
    
    h2o=TG_IR.ir['h2o']
    DTG=-sp.signal.savgol_filter(h2o, window_length, polyorder, deriv=1)
    d2TG=-sp.signal.savgol_filter(h2o, window_length, polyorder, deriv=2)
    
    peaks,properties=find_peaks(DTG,height=.0001,width=0,rel_height=1)#.5e-7
    end=properties['right_ips'].astype(np.int, copy=False)
    min_T=80
    max_T=250
    min_i=TG_IR.ir['Ts'][TG_IR.ir['Ts']>min_T].index[0]
    max_i=TG_IR.ir['Ts'][TG_IR.ir['Ts']>max_T].index[0]    

    dry_point=TG_IR.ir['t'][peaks[0]:][DTG[peaks[0]:]<0.000075].iloc[0]#TG_IR.ir['t'].iloc[step_end[0]]#max(step_end)
    
    #getting the dry_mass at the dry_point as well as the final weight and calculating the relative
    #mass-loss and the water content from it
    dry_mass=TG_IR.tga['mass'][dry_point]
    fin_weight=TG_IR.tga['mass'][len(TG_IR.tga)-1]
    mass_loss=(dry_mass-fin_weight)
    rel_mass_loss=mass_loss/dry_mass
    dry_time=TG_IR.tga['t'][dry_point]
    dry_temp=TG_IR.tga['Ts'][dry_point]
    water_content=(TG_IR.info['mass']-dry_mass)/dry_mass
    
    #write information in dictionary
    dry_info={}
    dry_info['dry_mass']=dry_mass
    dry_info['fin_mass']=fin_weight
    dry_info['ml_n2']=mass_loss
    dry_info['rel_ml_n2']=rel_mass_loss
    dry_info['dry_temp']=dry_temp
    dry_info['dry_time']=dry_time
    dry_info['water_content']=water_content
    
    #plotting
    if plot==True:
        fig=plt.figure()
        x=TG_IR.tga['Ts']
        y=TG_IR.tga['mass']
        plt.plot(x,y,label='TGA')
        #Trockenpunkt zeichnen
        plt.annotate(s='', xy=(x[dry_point],min(y)), xytext=(x[dry_point],y[dry_point]), arrowprops=dict(arrowstyle='<->'))
        plt.scatter(x[dry_point],y[dry_point],c='r')
        plt.hlines(fin_weight,x[dry_point],max(x),linestyle='dashed')
        plt.text(x[dry_point]+20,y[dry_point],'dry weight: '+str(round(y[dry_point],2))+' mg @ '+str(round(x[dry_point],2))+' °C')
        plt.text(x[dry_point]+20,(y[dry_point]+ fin_weight)/2,'mass loss: '+str(round(mass_loss,2))+' mg ('+str(round(rel_mass_loss*100,2))+' %)')
        plt.ylabel('TGA /mg')
        plt.xlabel('T /°C')
        ax2=plt.twinx()
        ax2.plot(TG_IR.ir['Ts'],h2o,label='h2o',linestyle='dashed')
        ax2.set_ylabel('DTG')
        plt.legend()
        plt.show()
        
    return dry_info

In [4]:
def read_TGA(file,profile='Otto'):
    #open file from TGA in given directory and make a DataFrame from it
    path=find_files(file,'.txt',dir_data)[0]
    
    if profile=='Otto':
        skiprows=13
        skipheader=7
        
    if profile=='Falk':
        skiprows=11
        skipheader=6
        
    try:
        data=pd.read_csv(path, delim_whitespace=True,decimal=',' ,names=['Index','t','Ts','Tr','mass'],skiprows=skiprows, skipfooter=11,converters={'mass':lambda x: float(x)}).drop(columns='Index')
        
    except:
        return
    
    #check if there is heat flow information and append it 
    try:
        path_mW=find_files(file,'_mW.txt',dir_data)[0]
        data['heat_flow']=pd.read_csv(path_mW, delim_whitespace=True,decimal=',' ,names=['Index','t','Ts','Tr','heat_flow'],skiprows=skiprows, skipfooter=11, usecols=['heat_flow'])
    except:
        pass
    
    #extract information on the measurement from the header and footer of the TGA file
    footer=pd.read_table(path,encoding='ansi',skipfooter=2,index_col=False,names=[0]).tail(3)
    header=pd.read_table(path,encoding='ansi',skiprows=skipheader,nrows=1,index_col=False,names=[0])
    
    header=str(header.iloc[0,0])
    name=re.search('\S+(?=,)',header).group()
    date=header[header.find(',')+2:header.rfind(' ')].strip()
    time=header[header.rfind(' ')+1:].strip()
    method_name=str(footer.iloc[2,0]).strip()
    mass=str(footer.iloc[0,0]).strip()
    mass=pd.to_numeric(mass[mass.find(',')+1:-3].replace(',','.'))
    
    #if the sample wasn't weighed in automatically, the mass at t=0 is used instead
    if mass==0:
        mass=data['mass'][0]
    

    # extract method info from method
    last_i=0
    values=[]
    parameters=[]
    for i in range(len(method_name)):
        if method_name[i]=='=':
            parameters.append('background_state')
            
        elif method_name[i]=='<':
            parameters.append('lower_temp')
            
        elif (method_name[i]=='>') or (method_name[i]=='/'):
            parameters.append('high_temp')
        
        elif (method_name[i]==')') and (method_name[last_i-1]=='('):
            parameters.append('method_gas')
            
        elif (method_name[i]=='(') and (method_name[last_i-1]=='/'):
            parameters.append('gradient')
            
        elif (method_name[i]=='/') and (method_name[last_i-1]=='<'):
            parameters.append('high_temp')
            
        if (method_name[i] in '=<>()/')==True:
            val=method_name[last_i:i]
            if val.isnumeric() ==True:
                val=int(val)
            values.append(val)
            last_i=i+1
    
    parameters.append('crucible')
    values.append(method_name[method_name.rfind('_')+1:])
    
    #write information in dictionary
    info={}
    info['name']=name
    #info['sample']=name[name.rfind('6.6_')+4:name.rfind('6.6_')+9]
    #info['sample']=re.search('\d{5}',name).group()
    try:
        sample=re.search('(?<=\d_)\w[^_]+(?=_\d{2,3})',name).group()
    except:
        sample=name
    try:
        info['sample']=int(sample)
    except:
        info['sample']=sample
    info['run']=re.search('(?<=_)\d{2,3}(?=_|$)',name).group()
    info['method_name']=method_name
    info['date']=date
    info['time']=time
    info['mass']=mass
    info['method_param']=parameters
    info['method_value']=values
    
    return data,info

def read_FTIR(file_name):
    files=find_files(file_name,'.csv',dir_data)
    #find the gases by looking at the suffix of the files
    gases=[]
    paths=[]
    for file in files:
        gases.append(file[file.rfind('_')+1:file.rfind('.')].lower())
        paths.append(file)  
    if gases==[]:
        return
    else:
        #make DataFrame with the first gas, keeping the time column
        data=pd.read_csv(files[0], delimiter=';', decimal=',', names=['t',gases[0]])
    
        #append the IR data from the other gases as new columns
        for i in range(1,len(gases)):
            data[gases[i]]=pd.read_csv(files[i], delimiter=';', decimal=',', names=['t',gases[i]], usecols=[gases[i]])
        data['t']=((data['t']+5)*60).astype(int)
        return data.dropna()

# Calibration

```massestufe``` takes TGA-data and calculates the mass steps. For that the first order derivative of the TG-values is used, DTG. The signal is evaluated for peaks and the peak-boundaries on either side are determined with the *find_peaks*-function from the *SciPy*-library. The right boundaries of the DTG-peaks mark the start of a mass step  and the height of the mass step is calculated as the mean TGA-value over a given number of samples, starting from said boundary.

In [10]:
def massestufe(TGA,rel_height=.98,plot=False): #rel_height=.963
    #Berechnung und Glättung von DTG
    window_length=201
    polyorder=3
    
    TG=(TGA['mass']/TGA['mass'][0])
    if window_length == 0:
        DTG=pd.Series(np.gradient(TG))
    else:
        DTG=sp.signal.savgol_filter(TG, window_length, polyorder, deriv=1)
  
    #Finden der Stufen
    peaks,properties=find_peaks(-DTG,height=0.00025,width=0,rel_height=rel_height)
    step_end=properties['right_ips'].astype(np.int, copy=False)
    step_start=properties['left_ips'].astype(np.int, copy=False)
    
    #Berechnen der Massen der Stufen
    stufen=np.zeros(len(peaks))
    samples=20
    for i in range(len(peaks)):
        stufen[i]=np.mean(TGA['mass'][step_end[i]:step_end[i]+samples])
    
    #Berechnen der Stufenhöhe
    stufenhöhe=np.zeros(len(stufen))
    stufen=np.insert(stufen,0,TGA['mass'][0])
        
    stufenhöhe=np.zeros(len(step_start))
    for i in range(len(step_start)):
        stufenhöhe[i]=np.mean(TGA['mass'][step_start[i]-samples:step_start[i]])-np.mean(TGA['mass'][step_end[i]:step_end[i]+samples])
    
    #print(stufenhöhe)
    rel_stufenhöhe=stufenhöhe/stufen[0]*100
    
    #Plotten
    if plot==True:
        #Plotten von DTG
        fig=plt.figure()
        x=TGA['Ts']
        y=-DTG
        plt.plot(x,y)
        plt.vlines(x[step_end],0,max(y),linestyle='dashed')
        plt.vlines(x[step_start],0,max(y),linestyle='dashed')
        plt.vlines(x[peaks],y[peaks]-properties['peak_heights'],y[peaks])
        plt.hlines(y[peaks]-properties['peak_heights'],x[step_end],x[step_start])
        plt.xlabel(x.name+' /°C')
        plt.ylabel('DTG /mg/°C')
        plt.title('DTG')
        plt.show()
        
        #Plotten der gefundenen Stufen
        fig=plt.figure()
        plt.hlines(stufen[:-1],np.zeros(len(stufen)),x[step_end],linestyle='dashed')
        plt.vlines(x[step_end],stufen[1:],stufen[:-1],linestyle='dashed')
        for i in range(len(step_end)):
            plt.text(x[step_end[i]]+5,stufen[i+1]+stufenhöhe[i]/2,str(round(stufenhöhe[i],3))+' mg ('+str(round(rel_stufenhöhe[i],2))+' %)')
        plt.plot(x,TGA['mass'])
        plt.xlabel(x.name+' /°C')
        plt.ylabel('mass /mg')
        plt.title('TG')
        plt.show()
        
        #Plotten der rel Stufen
        fig=plt.figure()
        rel_stufen=stufen/stufen[0]*100
        plt.hlines(rel_stufen[:-1],np.zeros(len(rel_stufen)),x[step_end],linestyle='dashed')
        plt.vlines(x[step_end],rel_stufen[1:],rel_stufen[:-1],linestyle='dashed')
        for i in range(len(step_end)):
            plt.text(x[step_end[i]]+5,rel_stufen[i+1]+rel_stufenhöhe[i]/2,str(round(rel_stufenhöhe[i],2))+' %')
        plt.plot(x,TGA['mass']/TGA['mass'][0]*100)
        plt.text(800,95,str(round(TGA['mass'][0],2))+' mg', horizontalalignment='center')
        plt.xlabel('T /°C')
        plt.ylabel('rel. mass /%')
        plt.title('TG')
        plt.show()
        
    return stufenhöhe,rel_stufenhöhe,step_start,step_end

```integrate_peaks``` takes FTIR-data and the mass step boundaries determined in ```massestufe``` and used them to integrate the gas-signals between the given boundaries and returns them in form of a table.

In [12]:
def integrate_peaks(FTIR,step_start,step_end,corr_baseline=None,plot=False):
    gases=['co','co2','h2o']
    
    #Umrechnen des Stufenstarts fürs FTIR. Zeitversatz durch Wartezeit und Transferline
    step_start=step_start+60
    step_end=step_end+60
   # print(step_start,step_end)
    integrals=pd.DataFrame(columns=[0,1,2],index=gases)
    
    #Plotten mit Integrationsgrenzen
    if plot==True:
        x=FTIR['t']
        y=FTIR[['co','co2','h2o']]
    
        fig, h2o=plt.subplots()
        fig.subplots_adjust(right=.8)
    
        co=h2o.twinx()
        co2=h2o.twinx()

        co2.spines['right'].set_position(('axes',1.15))
    
        gco,=co.plot(x,FTIR['co'],'r-',label='CO')
        gco2,=co2.plot(x,FTIR['co2'],'b-',label='CO2')
        gh2o,=h2o.plot(x,FTIR['h2o'],'g-',label='H2O')
        co2.vlines(step_end,0,max(FTIR['co2']),linestyle='dashed')
        co2.vlines(step_start,0,max(FTIR['co2']),linestyle='dashed')
    
        h2o.set_xlabel('t /Min')
        h2o.set_ylabel('H2O')
        co.set_ylabel('CO')
        co2.set_ylabel('CO2')
        
        h2o.yaxis.label.set_color(gh2o.get_color())
        co.yaxis.label.set_color(gco.get_color())
        co2.yaxis.label.set_color(gco2.get_color())
    
        graphs=[gco,gco2,gh2o]
    
        h2o.legend(graphs,[g.get_label() for g in graphs])
        plt.title('Absorbance')
    
    #Integration
    for gas in gases:
        for i in range(len(step_end)):
            subset=FTIR[gas][(FTIR['t']>=step_start[i]) & (FTIR['t']<=step_end[i])]
            #Baseline correction
            if corr_baseline=='linear':
                baseline=np.linspace(subset.iloc[0],subset.iloc[len(subset)-1],len(subset))
            elif corr_baseline=='const':
                baseline=min(subset.iloc[0],subset.iloc[len(subset)-1])*np.ones(len(subset))#np.linspace(min(subset),min(subset),len(subset))
            elif corr_baseline==None:
                baseline=np.zeros(len(subset))
                
            integral=sp.integrate.simps(subset-baseline)   
            integrals[i][gas]=integral
            
            if plot==True:
                    x=FTIR['t'][(FTIR['t']>=step_start[i]) & (FTIR['t']<=step_end[i])]
                    y=subset
                    if gas=='co':
                        co.plot(x,baseline,'r--')
                    if gas=='co2':
                        co2.plot(x,baseline,'b--')
                    if gas=='h2o':
                        h2o.plot(x,baseline,gh2o.get_color())
    if plot==True:
        plt.show()
        print('Baseline correction: ',corr_baseline)
        
    return integrals 
        

```eval_lin``` returns the evaluation of a linear function $y=a\cdot x +b$ with slope $a$ and intersect $b$ at a given point.

In [13]:
def eval_lin(x,slope,intercept):
    return slope*x+intercept

```calibration_stats``` calculates the detection limit $x_{NG}$ or $LOD$, limit of detection and limit of determination $x_{BG}$ or $LOQ$ based on *DIN 32645* in *mmol* as well as the residual and process standard deviation $s_{yx}$ and $s_{x0}$ in *FE* and *mmol*, respectively. Confidence is by default $\alpha=.95=\beta$, $k=3$ for the uncertainty of results $\frac1k=.33$  and the number of analysis for the sample is $m=1$

In [14]:
def calibration_stats(x_Kali,y_Kali,linreg,alpha=.95,beta=None,m=1,k=3):
    gases=linreg.index
    
    n=len(x_Kali)
    if n==2:
        return pd.DataFrame()
    f=n-2
    if beta==None:
        beta=alpha
    
    stats=pd.DataFrame(columns=['s_yx','s_x0','x_NG','x_EG','x_BG'])
    for gas in gases:
        b=linreg['slope'][gas]
        a=linreg['intercept'][gas]
        
        s_yx=np.sqrt(sum(np.power(b*x_Kali[gas]+a-y_Kali[gas],2))/(n-2))
        s_x0=s_yx/b
        x_=np.mean(x_Kali[gas])
        Q_x=sum(np.power(x_Kali[gas]-x_,2))
        
        x_NG=s_x0*sp.stats.t.ppf(alpha,f)*np.sqrt(1/m+1/n+(x_*x_)/Q_x)
        
        x_EG=x_NG+s_x0*sp.stats.t.ppf(beta,f)*np.sqrt(1/m+1/n+(x_*x_)/Q_x)
    
        x_BG=k*x_NG
                
        stats=stats.append(pd.DataFrame([[s_yx,s_x0,x_NG,x_EG,x_BG]],index=[gas],columns=['s_yx','s_x0','x_NG','x_EG','x_BG']))
    print('s_yx in FE\ts_x0 in mmol\tx_NG in mmol\tx_EG in mmol\tx_BG in mmol')
    return stats

```calibrate``` uses the samples and baseline measurements given in the file "Kali.txt" to calibrate the FTIR.
The overall procedure contains of the following steps:
1. correct the TGA and FTIR-data with the according functions
2. calculate the mass steps
3. calculate the integrals for each gas for each mass step
4. calculate the theoretical amount of substance by dividing the according mass step by the molar mass
5. appending the amount of substance and the integral to a dataset
6. calculate the linear regression from the integral over the amount of substance for $CO_2$ and $H_{2}O$
7. correct the amount of substance of $CO$ by the amount that is consumed in the water-gas shift reaction
8. calculate the linear regression from the integral over the amount of substance for $CO$
9. return the parameters of the regression in form of a table
10. (optional) visualization

In [15]:
def calibrate(plot=False,sample_list=None):
    Stoffdaten=pd.DataFrame({'co2':44.01,'co':28.01,'h2o':18.02,'caox':128.10},index=['M'])
    gases=['h2o','co','co2'] 
    
    #gespeicherte Kalibrierung laden
    os.chdir(dir_cali)
    try:
        cali=pd.read_excel('cali.xlsx',sheet_name=None,index_col=0)
        linreg=cali['linreg']
        x_Kali=cali['x']
        y_Kali=cali['y']
        stats=cali['stats']
        print('Hinterlegte Kalibrierung geladen.')
        print(linreg)
        
    #neu kalibrieren
    except:
        print('Kalibriere...')
        #Einlesen der Samplenamen und Baselines, die zur Kalibration genutzt werden sollen
        samples=pd.read_csv(os.path.join(dir_samples,sample_list+'.txt'),delimiter='\t')
        
        #initialisieren der DataFrames für x und y-werte
        x_Kali=pd.DataFrame(columns=gases)
        y_Kali=pd.DataFrame(columns=gases+['co_corr','co2_corr'])
        
        #Berechnen der Massenstufen und Integrale der FTIR-Signale für alle Proben
        linreg=pd.DataFrame(columns=['slope','intercept','r_value','p_value','std_error'])
        for i in range(len(samples)):
            #Einlesen der Daten
            path=samples['Samples'][i]
            path_baseline=samples['Baseline'][i]

            TGA,sample_info=read_TGA(path)
            if sample_info['sample']!=18358:
                print('Wrong sample for calibration.')
                break
            TGA=corr_TGA(TGA,path_baseline)
            FTIR=read_FTIR(path)
            FTIR=corr_FTIR(FTIR,path_baseline,plot=True)
            
            #update_samplelog(sample_info)
            print('----------------------------------------------------')
            print(sample_info)

            #Berechnen der Massestufen und integration der Freistzungsraten
            [steps,rel_steps,stepstart,stepend]=massestufe(TGA,plot=False)
            integrals=integrate_peaks(FTIR,stepstart,stepend,plot=False,corr_baseline=None)

            #Einfügen der x-Werte (/mg) und y-Werte(/FE)
            x_Kali=x_Kali.append({'co2':steps[2],'co':steps[1],'h2o':steps[0]},ignore_index=True)
            y_Kali=y_Kali.append({'co2':integrals[2]['co2'],'co':integrals[1]['co'],'h2o':integrals[0]['h2o'],'co_corr':integrals[1]['co2'],'co2_corr':integrals[2]['co']},ignore_index=True)

        #Lineare Regression
        for gas in gases:
            slope, intercept, r_value, p_value, std_err=sp.stats.linregress(x_Kali[gas],y_Kali[gas])
            linreg=linreg.append(pd.DataFrame([[slope,intercept,r_value,p_value,std_err]],index=[gas],columns=['slope','intercept','r_value','p_value','std_error']))
            plt.scatter(x_Kali[gas],y_Kali[gas])
            plt.plot(x_Kali[gas],slope*x_Kali[gas]+intercept)
            plt.text(0,0,str(slope))
            plt.show()

        #Korrektur        
        colors =plt.rcParams['axes.prop_cycle'].by_key()['color']
        fig=plt.figure()
        fig.set_size_inches(15,20)
        n_iter=10
        for i in range(n_iter):
            #print('---------------------------\n >> Iteration: ',i)
            #Korrektur der CO2 masse um co masse im selben zersetzungsschritt
            corr=(y_Kali['co2_corr']-linreg['intercept']['co'])/linreg['slope']['co']
            temp=np.subtract(x_Kali['co2'],corr*(corr >0))#stats['x_BG']['co']))
            
            #neue regression
            slope, intercept, r_value, p_value, std_err=sp.stats.linregress(temp,y_Kali['co2'])
            
            #zwischenspeichern
            co2=pd.DataFrame([[slope,intercept,r_value,p_value,std_err]],index=['co2'],columns=['slope','intercept','r_value','p_value','std_error'])
            
            plt.subplot(211)
            plt.title('co2')
            plt.scatter(temp,y_Kali['co2'],color=colors[i],label='it. '+str(i+1))
            plt.plot(temp,temp*slope+intercept,color=colors[i])
            
            #Korrektur der CO masse um co2 masse im selben zersetzungsschritt            
            corr=(y_Kali['co_corr']-linreg['intercept']['co2'])/linreg['slope']['co2']
            temp=np.subtract(x_Kali['co'],corr*(corr>0))#stats['x_BG']['co2']))
            
            #neue regression
            slope, intercept, r_value, p_value, std_err=sp.stats.linregress(temp,y_Kali['co'])
 
            #zwischenspeichern
            co=pd.DataFrame([[slope,intercept,r_value,p_value,std_err]],index=['co'],columns=['slope','intercept','r_value','p_value','std_error'])
                        
            plt.subplot(212)
            plt.title('co')
            plt.scatter(temp,y_Kali['co'],color=colors[i],label='it. '+str(i+1))
            plt.plot(temp,temp*slope+intercept,color=colors[i])
            
            linreg.update(co2)
            linreg.update(co)
            
            stats=calibration_stats(x_temp,y_Kali,linreg)
            
        x_Kali['co']=x_Kali['co']-(y_Kali['co_corr']-linreg['intercept']['co2'])/linreg['slope']['co2']
        x_Kali['co2']=x_Kali['co2']-(y_Kali['co2_corr']-linreg['intercept']['co'])/linreg['slope']['co']
        
        for gas in gases:
            x_Kali[gas].update(x_Kali[gas]/Stoffdaten[gas]['M'])
            
            slope, intercept, r_value, p_value, std_err=sp.stats.linregress(x_Kali[gas],y_Kali[gas])
        
            temp=pd.DataFrame([[slope,intercept,r_value,p_value,std_err]],index=[gas],columns=['slope','intercept','r_value','p_value','std_error'])
            linreg.update(temp)
            
        plt.legend()
        plt.show()
        stats=calibration_stats(x_Kali,y_Kali,linreg)
        
        #speichern der Kalibrierung
        with pd.ExcelWriter('cali.xlsx') as writer:
            linreg.to_excel(writer,sheet_name='linreg')
            x_Kali.to_excel(writer,sheet_name='x')
            y_Kali.drop(['co_corr','co2_corr'],axis=1).to_excel(writer,sheet_name='y')
            stats.to_excel(writer,sheet_name='stats')
    
    #Plotten der Regressionsgeraden und der zugrunde liegenden Daten
    if plot:
        for gas in gases:
            fig=plt.figure()
            x=x_Kali[gas]
            y=y_Kali[gas]
            
            plt.scatter(x,y,label='data (N = {})'.format(len(x)))
            
            x=np.array((min(x),max(x)))
            plt.plot(x,x*linreg['slope'][gas]+linreg['intercept'][gas],label='regression',ls='dashed')
            plt.text(max(x),min(y),'y = {:.3f} $\cdot$ x {:+.3f}, $R^2$ = {:.5}'.format(linreg['slope'][gas],linreg['intercept'][gas],str(linreg['r_value'][gas])),horizontalalignment='right')
            plt.xlabel('mmol')
            plt.ylabel('AU')
            plt.title(get_label(gas))
            plt.xlim(0,max(x)+abs(min(x)))
            plt.legend(loc=0)
            plt.show()
            
        fig=plt.figure()
        for gas in gases:
            x=x_Kali[gas]
            y=y_Kali[gas]
            
            plt.scatter(x,y,label='data {} (N = {})'.format(get_label(gas),len(x)))
            
            x=np.array((min(x),max(x)))
            plt.plot(x,x*linreg['slope'][gas]+linreg['intercept'][gas],label='regression {}'.format(get_label(gas)),ls='dashed')
            plt.xlabel('mmol')
            plt.ylabel('AU')
            plt.xlim(0,max(x)+abs(min(x)))
            plt.legend(loc=0)
        plt.show()
    os.chdir(home)
    return linreg,stats

# Fitting

```gaussian``` takes the parameters of the gaussian function and returns its value at a given point x:

$g(x)=height\cdot \exp(-\ln(2)\cdot (\frac{x-center}{hwhm})^2)$

In [16]:
def gaussian(x,height,center,hwhm):
    return height*np.exp(-np.log(2)*np.power((x-center)/hwhm,2))

```multi_gauss``` takes lists of parameters of the normal distribution and returns the sum of those distrubionts at a given point x:
$\sum_{i}^{n}g_{i}(x)$

In [17]:
def multi_gauss(x,*args):
    n=int(len(args)/3)
    heights=args[:n]
    centers=args[n:2*n]
    hwhms=args[2*n:len(args)]
    
    s=0
    for i in range(n):
        s=s+gaussian(x,heights[i],centers[i],hwhms[i])
    return s

```fitting``` takes FTIR-data, the gas to fit and a function to fit the data and returns the optimal parameters for the given function with the least sum of squares. The fit-function from the *SciPy*-library uses the *Levenberg-Marquardt* algorithm.

In [18]:
def fitting(TG_IR,centers,labels,func,tol_center=30,max_hwhm=95,y_axis='orig',plot=False,save=True,init_offs=[0,0,0]):
    gases=list(centers.columns.values)
    if ('co2' in gases) and ('co' in gases):
        gases.remove('co2')
        gases.insert(0,'co2')
    #thresholds for fit parameters
    
    #initializing output DataFrame
    peaks=pd.DataFrame(columns=['center','height','hwhm','area','mmol','mmol_per_g'],index=[group+'_'+gas.upper() for gas in gases for group in labels[gas].dropna()]+[gas.upper() for gas in gases])
    sumsqerr=pd.DataFrame(index=[TG_IR.info['name']],columns=gases)

    #cycling through gases
    FTIR=TG_IR.ir.copy()
    for gas in gases:
        #print(gas)
        params=pd.DataFrame(columns=['min_center','init_center','max_center','min_height','init_height','max_height','min_hwhm','init_hwhm','max_hwhm'],index=[group+'_'+gas.upper() for group in labels[gas].dropna()])

        if gas=='h2o':
            FTIR[gas]-=baseline_als(FTIR[gas])
            
        #molar desorption
        tot_area=np.sum(TG_IR.ir[gas])
        if gas == 'h2o':
            tot_area=np.sum(TG_IR.ir[gas][TG_IR.ir['Ts']>150])
        tot_mol=(tot_area-TG_IR.linreg['intercept'][gas])/TG_IR.linreg['slope'][gas]
        peaks['area'][gas.upper()]=tot_area
        peaks['mmol'][gas.upper()]=tot_mol
        peaks['mmol_per_g'][gas.upper()]=tot_mol/TG_IR.info['dry_mass']*1000
        

        
        if y_axis=='rel':
            FTIR.update(FTIR[gas]/tot_area*tot_mol)
        
        #initial guesses
        init_centers=np.array(centers[gas].dropna().astype(float))+init_offs[0]
        num_curves=len(init_centers)
        max_heigth=max(FTIR[gas])
        init_heights=(0.5+init_offs[1])*max_heigth*np.ones(num_curves)#.5*TG_IR.info['dry_mass']/200*np.ones(num_curves)#
        init_hwhms=(0.5+init_offs[2])*max_hwhm*np.ones(num_curves)#0.5*max_hwhm
        
        #lower bounds for fit
        min_heights=0*np.ones(num_curves)
        min_hwhms=0*np.ones(num_curves)
        min_centers=init_centers-tol_center
        
        #upper bounds for fit
        max_heights=max_heigth*np.ones(num_curves)#init_heights+5*TG_IR.info['dry_mass']/200##
        max_hwhms=max_hwhm*np.ones(num_curves)
        max_centers=init_centers+tol_center
        
        #print(min_centers,init_centers,max_centers)
        params['init_center']=init_centers
        params['init_height']=init_heights
        params['init_hwhm']=init_hwhms
        params['min_center']=min_centers
        params['min_height']=min_heights
        params['min_hwhm']=min_hwhms
        params['max_center']=max_centers
        params['max_height']=max_heights
        params['max_hwhm']=max_hwhms
        #assumptions by figueiredo
        a=True
        if (gas=='co' and a==True):
            try:
                loc=labels[gas][labels[gas]=='anhydrides'].index[0]

                init_heights[loc]=peaks['height']['anhydrides_CO2']/TG_IR.linreg['slope']['co2']*TG_IR.linreg['slope']['co']
                min_heights[loc]=init_heights[loc]*.9
                max_heights[loc]=init_heights[loc]*1.1

                init_hwhms[loc]=peaks['hwhm']['anhydrides_CO2']
                min_hwhms[loc]=init_hwhms[loc]*.9
                max_hwhms[loc]=init_hwhms[loc]*1.1
            except:
                pass
            
            
        #guesses and bounds
        init_guess=np.concatenate((init_heights,init_centers,init_hwhms))
        min_param=np.concatenate((min_heights,min_centers,min_hwhms))
        max_param=np.concatenate((max_heights,max_centers,max_hwhms))
    
        #actual fitting
        x=FTIR['Ts']
        popt,pcov=curve_fit(func,x,FTIR[gas],p0=init_guess,bounds=(min_param,max_param))
        
        #return values
        for i in range(num_curves):
            group=labels[gas][i]+'_'+gas.upper()
            peaks['height'][group]=popt[i]
            peaks['center'][group]=popt[i+num_curves]
            peaks['hwhm'][group]=popt[i+2*num_curves]
            if y_axis=='orig':
                peaks['area'][group]=np.sum(gaussian(x,popt[i],popt[i+num_curves],popt[i+2*num_curves]))
                peaks['mmol'][group]=peaks['area'][group]/tot_area*tot_mol
                peaks['mmol_per_g'][group]=peaks['mmol'][group]*1000/TG_IR.info['dry_mass']#np.power(2*np.pi,0.5)*peaks['height'][labels[gas][i]+'('+gas+')']*peaks['hwhm'][labels[gas][i]+'('+gas+')']/(np.power(2*np.log(2),0.5))
            elif y_axis=='rel':
                peaks['mmol'][group]=peaks['area'][group]/tot_area*tot_mol
                peaks['mmol_per_g'][group]=peaks['mmol'][group]*1000/TG_IR.info['dry_mass']
        ###plotting
        
        profiles=pd.DataFrame()
        data=FTIR[gas]
        fit=multi_gauss(x,*popt)
        diff=data-fit
        sumsqerr[gas][TG_IR.info['name']]=np.sum(np.power(diff,2))
        profiles['Ts']=x
        profiles['data']=data
        profiles['fit']=fit
        profiles['diff']=diff
            
        if plot==True:
            #setup plot
            fig=plt.figure(constrained_layout=True)
            gs = fig.add_gridspec(8, 1)
            fitting = fig.add_subplot(gs[:-1, 0])
            fitting.set_title('{} - {}, {} mg'.format(get_label(TG_IR.info['sample']),TG_IR.info['run'],TG_IR.info['mass']))
            error = fig.add_subplot(gs[-1,0],sharex=fitting)
            fitting.xaxis.set_ticks(np.arange(0, 1000, 50))
            
            #plotting of fit
            fitting.plot(x,data,label='data',lw=2,zorder=num_curves+1)#,ls='',marker='x',markevery=2,c='cyan')
            fitting.plot(x,fit,label='fit',lw=2,zorder=num_curves+2)
        for i in range(0,num_curves):
            y=gaussian(x,popt[i],popt[i+num_curves],popt[i+2*num_curves])
            profiles[labels[gas][i]]=y
            if plot==True:
                fitting.text(popt[num_curves+i],popt[i],labels[gas][i],zorder=num_curves+3+i)
                fitting.plot(x,y,linestyle='dashed',zorder=i)
        if plot==True:
            fitting.legend()
            fitting.set_xlabel('T /°C')
            if y_axis=='orig':
                fitting.set_ylabel('{} /AU'.format(get_label(gas)))
            elif y_axis=='rel':
                fitting.set_ylabel('{} /mmol/g/s'.format(get_label(gas)))

            #mark center on x-axis
            fitting.scatter(popt[num_curves:2*num_curves],np.zeros(num_curves),marker=7,color='k',s=100,zorder=num_curves+3)

            #plotting of absolute difference
            abs_max=0.05*max(data)
            
            error.text(0,abs_max,'SQERR: {:.2e}'.format(sumsqerr[gas][TG_IR.info['name']]))#,'SQERR: '+'%.2E'% Decimal(sumsqerr[gas][TG_IR.info['name']]),va='bottom')
            error.plot(x,diff)
            error.hlines(0,min(x),max(x),ls='dashed')
            error.set_xlabel('T /°C')
            error.set_ylabel('error')#('AU')
            error.set_ylim(-abs_max,abs_max)
            plt.show()
            fig.savefig(TG_IR.info['name']+'_'+gas+'.png', bbox_inches='tight', dpi=300)
        if save==True:
            try:
                with pd.ExcelWriter(TG_IR.info['name']+y_axis+'.xlsx',engine='openpyxl', mode='a') as writer:
                    profiles.to_excel(writer,sheet_name=gas)
                    params.to_excel(writer,sheet_name=gas+'_param')
            except:
                with pd.ExcelWriter(TG_IR.info['name']+y_axis+'.xlsx',engine='openpyxl') as writer:
                    profiles.to_excel(writer,sheet_name=gas)
                    params.to_excel(writer,sheet_name=gas+'_param')
    if save==True:                
        with pd.ExcelWriter(TG_IR.info['name']+y_axis+'.xlsx',engine='openpyxl', mode='a') as writer:
                peaks.astype(float).to_excel(writer,sheet_name='summary')
    return peaks.astype(float),sumsqerr

```class TG_IR``` combines all functions in an object.

```TG_IR('name')``` Initializes the object loads the data from TGA and FTIR with given 'name'. 

```.corr('reference')``` Corrects the loaded data with a given 'reference' measurement.

```.plot(args)```   Visualizes the loaded data in different ways.

```.fit(args)``` Deconvolutes the data and quantifies the amount of SOG with uncertainties.

```.save()``` Saves the object as a pickle file. The file can be reloaded with ```load_TG_IR('name')```.

Data from multiple objects can be visualized in a common graph wit ```plots(*objs,args)```.

To perform the deconvolution procedure on multiple samples and summarize the data ```fits(*objs,args)``` can be used.

In [ ]:
class TG_IR:
    linreg,stats = calibrate(sample_list='Kali_neu')
    window_length=201
    polyorder=3
    def __init__(self,name,profile='Otto'):
        try:
            self.tga, self.info=read_TGA(name,profile=profile)
            self.info['dry_mass']=self.info['mass']
            self.tga['dtg']=-sp.signal.savgol_filter(self.tga['mass'],self.window_length,self.polyorder,deriv=1)
        except:
            print('No TG data for '+name+' was found')
        
        try:
            self.ir=read_FTIR(name)
            self.gases=self.ir.columns[1:]
        except:
            print('No IR data for '+name+' was found')
        try:
            self.ir=pd.merge(self.tga.filter(['t','Ts'],axis=1),self.ir, how='left',on='t').dropna(axis=0)
        except:
            pass
        
    def corr(self,baseline,which='both',plot=False):
        self.info['baseline']=baseline

        if which == 'both':
            self.tga=corr_TGA(self.tga,baseline,plot=plot)
            self.ir=corr_FTIR(self.ir,baseline,plot=plot)
            self.tga['dtg']=-sp.signal.savgol_filter(self.tga['mass'],self.window_length,self.polyorder,deriv=1)
            
        elif which == 'IR':
            self.ir=corr_FTIR(self.ir,baseline,plot=plot)
            
        elif which == 'TGA':
            self.tga=corr_TGA(self.tga,baseline,plot=plot)
            self.tga['dtg']=-sp.signal.savgol_filter(self.tga['mass'],self.window_length,self.polyorder,deriv=1)
            
        if which == 'both' or 'TGA':
            try:
                self.info.update(dry_weight(self,plot=plot))
            except:
                print('>> Trocknungsdaten konnten nicht erzeugt werden!')
                
                
    def plot(self,which,x_axis='Ts',y_axis='orig',gases=[],save=False):
        if which=='IR':
            plot_FTIR(self,x_axis=x_axis,y_axis=y_axis,gases=gases,save=save)
        
        if which=='DIR':
            temp=copy.deepcopy(self)
            temp.ir.update(self.ir.filter(self.gases,axis=1).diff().ewm(span = 10).mean())
            plot_FTIR(temp,x_axis=x_axis,y_axis=y_axis,gases=gases,save=save)
            
        if which=='D2IR':
            temp=copy.deepcopy(self)
            temp.ir.update(self.ir.filter(self.gases,axis=1).diff().diff().ewm(span = 20).mean())
            plot_FTIR(temp,x_axis=x_axis,y_axis=y_axis,gases=gases,save=save)
            
        if which=='TG':
            plot_TGA(self,'mass',x_axis=x_axis,y_axis=y_axis,save=save)
            
        if which=='heat_flow':
            plot_TGA(self,which,x_axis=x_axis,y_axis=y_axis,save=save)
            
        if which=='IR_to_DTG':
            FTIR_to_DTG(self,x_axis=x_axis,save=save,gases=gases)
        
        if which=='cumsum':
            temp=copy.deepcopy(self)
            temp.ir.update(self.ir.filter(self.gases,axis=1).cumsum())
            plot_FTIR(temp,x_axis=x_axis,y_axis=y_axis,gases=gases,save=save)
        
    def fit(self,reference,tol_c=25,tol_hwhm=75,T_max=None,plot=True,func=multi_gauss,y_axis='orig',save=True):
        if T_max==None:
            T_max=max(self.tga['Ts'])
        elif T_max>max(self.tga['Ts']):
            print('$T_{max}$ exceeds maximum temperature of data')
            T_max=max(self.tga['Ts'])
            
        ###extracting initial values for fitting from reference
        references=pd.read_excel(os.path.join(dir_parameters,'surfgr_C.xlsx'),index_col=0,header=None)
        init_params=references.loc[['gas',reference]].dropna(axis=1)

        #find needed gases
        gases=list(set(init_params.loc['gas']))

        #initial center
        temps=pd.DataFrame(columns=gases,index=range(len(init_params.columns)))
        
        #labels for the center
        labels=pd.DataFrame(columns=gases,index=range(len(init_params.columns)))
        for column in init_params.columns:
            temps[init_params[column]['gas']][temps[init_params[column]['gas']].count()]=init_params[column][reference]
            labels[init_params[column]['gas']][labels[init_params[column]['gas']].count()]=references[column]['group']
        temps=temps.where(temps<T_max+tol_hwhm).dropna(thresh=1)
        labels=labels.where(temps<T_max+tol_hwhm).dropna(thresh=1)
        
        if save==True:
            path=os.path.join(dir_fits,time()+reference+'_'+self.info['name'])
            os.mkdir(path)
            os.chdir(path)
            
        temp=copy.deepcopy(self)
        temp.tga=temp.tga[temp.tga['Ts']<T_max]
        temp.ir=temp.ir[temp.ir['Ts']<T_max]
        peaks, sumsqerr=fitting(temp,temps,labels,func,tol_center=tol_c,max_hwhm=tol_hwhm,plot=plot,y_axis=y_axis,save=save)
        
        os.chdir(home)
        return peaks, sumsqerr
    
    def save(self):
        with open(os.path.join(dir_output,self.info['name']+'.pkl'),'wb') as output:
            pickle.dump(self,output,pickle.HIGHEST_PROTOCOL)

def plots(*TG_IR,plot,x_axis='Ts',y_axis='orig',gas=None,save=False):
    fig=plt.figure()
    #out=pd.DataFrame()
    
    
    if x_axis=='t':
        plt.xlabel('t /s')
    elif x_axis=='Ts':
        plt.xlabel('T /°C')
    
    if plot=='TG':
        if y_axis=='orig':
            plt.ylabel('mass /mg')
        if y_axis=='rel':
            plt.ylabel('rel mass /%DM')
            
    if plot=='heat flow':
        if y_axis=='orig':
            plt.ylabel('heat flow /mW')
        if y_axis=='rel':
            plt.ylabel('heat flow /mWmg^{-1}')
            
    elif plot=='IR':
        if gas==None:
            gas=TG_IR[0].gases[0]
        if y_axis=='orig':
            plt.ylabel('{} /AU'.format(get_label(gas)))
        if y_axis=='rel':
            plt.ylabel('{} $/mmol g^{{-1}}s^{{-1}}$'.format(get_label(gas)))
            
    for obj in TG_IR:
        if plot=='TG':
            x=obj.tga[x_axis]
            if y_axis=='orig':
                y=obj.tga['mass']
            elif y_axis=='rel':
                y=obj.tga['mass']/obj.info['dry_mass']
            plt.plot(x,y,label=get_label(obj.info['sample'])+' - '+obj.info['run'])
        if plot=='heat flow':
            x=obj.tga[x_axis]
            if y_axis=='orig':
                y=obj.tga['heat_flow']
            elif y_axis=='rel':
                y=obj.tga['heat_flow']/obj.info['dry_mass']
            plt.plot(x,y,label=get_label(obj.info['sample'])+' - '+obj.info['run'])
        if plot=='IR':
            x=obj.ir[x_axis]
            if y_axis=='orig':
                y=obj.ir[gas]
                plt.plot(x,y,label=get_label(obj.info['sample'])+' - '+obj.info['run'])#+', dry mass: '+str(round(obj.info['dry_mass'],2))+' mg'
            elif y_axis=='rel':
                y=1000*obj.ir[gas]/obj.linreg['slope'][gas]/obj.info['dry_mass']*10/(obj.info['method_value'][obj.info['method_param'].index('gradient')])          
                plt.plot(x,y,label=r'{} - {}, {:.2f} mg'.format(get_label(obj.info['sample']),obj.info['run'],obj.info['mass']))
            
        
    plt.legend()
    plt.show()
    
    if save==True:
        fig.savefig(os.path.join(dir_plots,'_'.join(list(set([str(obj.info['sample']) for obj in TG_IR]))+[plot,gas,y_axis])+'.png'), bbox_inches='tight', dpi=300)
        

def fits(*TG_IR,reference=None,tol_c=30,tol_hwhm=95,T_max=None,plot=True,y_axis='orig',init_offs=[0,0,0],save=True,labels=None,temps=None):
    if reference!=None:
        references=pd.read_excel(os.path.join(dir_parameters,'surfgr_C.xlsx'),index_col=0,header=None)
        if T_max==None:
            T_max=min([max(obj.tga['Ts']) for obj in TG_IR])

        elif T_max>min([max(obj.tga['Ts']) for obj in TG_IR]):
            T_max=T_max=min([max(obj.tga['Ts']) for obj in TG_IR])
            print('$T_{max}$ exceeds maximum temperature of data')


        init_params=references.loc[['gas',reference]].dropna(axis=1)
        #find needed gases
        gases=list(set(init_params.loc['gas']))

        ###extracting initial values for fitting from reference
        #initial center
        temps=pd.DataFrame(columns=gases,index=range(len(init_params.columns)))
        #labels for the center
        labels=pd.DataFrame(columns=gases,index=range(len(init_params.columns)))
        for column in init_params.columns:
            temps[init_params[column]['gas']][temps[init_params[column]['gas']].count()]=init_params[column][reference]
            labels[init_params[column]['gas']][labels[init_params[column]['gas']].count()]=references[column]['group'] 
        temps=temps.where(temps<T_max+tol_hwhm).dropna(thresh=1)
        labels=labels.where(temps<T_max+tol_hwhm).dropna(thresh=1)

    else:
        gases=labels.columns
    
    #initializing of output DataFrames
    col_labels=[group+'_'+gas.upper() for gas in gases for group in labels[gas].dropna()]+[gas.upper() for gas in gases]
    err=pd.DataFrame(columns=gases)
    names=['center','height','hwhm','area','mmol','mmol_per_g']
    res=dict()
    for name in names:
        res[name]=pd.DataFrame(columns=col_labels)
    
    #make subdirectory to save data
    if save==True:
        path=os.path.join(dir_fits,time()+reference+'_'+'_'.join(list(set([str(obj.info['sample']) for obj in TG_IR]))))
        os.mkdir(path)
        os.chdir(path)
    
    #cycling through samples
    for obj in TG_IR:
        #fitting of the sample and calculating the amount of functional groups
        if T_max!=None:
            temp=copy.deepcopy(obj)
            temp.tga=temp.tga[temp.tga['Ts']<T_max]
            temp.ir=temp.ir[temp.ir['Ts']<T_max]
        else:
            temp=obj
        peaks,sumsqerr=fitting(temp,temps,labels,multi_gauss,tol_center=tol_c,max_hwhm=tol_hwhm,plot=plot,y_axis=y_axis,save=save,init_offs=init_offs)

        #writing data to output DataFrames
        for key in res:
            res[key]=res[key].append(peaks[key].rename(obj.info['name']).T)  
        err=err.append(sumsqerr)

    # calculate statistical values
    dm=1e-6
    for key in res:
        samples=list(set([get_label(re.search('(?<=_)\d{5}(?=_\d{2,3})',index).group()) for index in res[key].index]))
        stddev=pd.DataFrame(columns=res[key].columns,index=[sample+'_stddev' for sample in samples])
        mean=pd.DataFrame(columns=res[key].columns,index=[sample+'_mean' for sample in samples])
        for sample in samples:
            for column in res[key].columns:
                gas=column[column.rfind('_')+1:].lower()
                indices=[index for index in res[key].index if get_label(re.search('(?<=_)\d{5}(?=_\d{2,3})',index).group())==sample]
                subset=res[key][column].loc[indices]
                if key=='mmol_per_g':
                    mmol=res['mmol'][gas.upper()].loc[indices]#res['mmol'][column].loc[indices]
                    g=mmol/subset
                    lod=TG_IR[0].stats['x_NG'][gas]
                    dmmolg_i=np.power(np.power(lod/mmol,2)+np.power(dm/g,2),0.5)*subset
                    dmmol=np.power(np.sum(np.power(dmmolg_i,2)),0.5)
                    stddev[column][sample+'_stddev']=dmmol
                else:
                    stddev[column][sample+'_stddev']=np.std(subset)
                mean[column][sample+'_mean']=np.mean(subset)
        res[key]=res[key].append(mean)
        res[key]=res[key].append(stddev)        
    
    #exporting data
    if save==True:
        with pd.ExcelWriter('summary.xlsx') as writer:
            for key in res:
                res[key].dropna(axis=1).to_excel(writer,sheet_name=key)
            err.to_excel(writer,sheet_name='sum_squerr')
        os.chdir(home)
    return res

def read_TG_IR(name):
    with open(os.path.join(dir_output,name+'.pkl'), 'rb') as inp:
        obj = pickle.load(inp)
    return obj